Really useful [link](https://dev.indooroutdoor.io/asyncio-demystified-rebuilding-it-from-scratch-one-yield-at-a-time)

**Generator**

Generators are functions that can pause and resume their execution.

When a generator function is called, it returns a generator object, which is an iterator.

The code inside the function is not executed yet, it is only compiled. The function only executes when you iterate over the generator.

The `yield` keyword is what makes a function a generator.

When `yield` is encountered, the function's state is saved, and the value is returned. The next time the generator is called, it continues from where it left off.

In [89]:
from time import sleep

def fibonnaci_generator():
    n_1 = 0
    n_2 = 1
    while True:
        n = n_1 + n_2
        yield n
        sleep(.5)

        n_2 = n_1
        n_1 = n

fib_gen = fibonnaci_generator()
print(fib_gen)

<generator object fibonnaci_generator at 0x105442c20>


In [90]:
for x in fib_gen:
    print(x)
    if x > 100:
        break

1
1
2
3
5
8
13
21
34
55
89
144


In [91]:
def filter_even():
    while True:
        x = yield
        print(x)
        if not x:
            continue
        if x % 2 == 0:
            print(x)
even_filter = filter_even()
next(even_filter)  # Start the generator

In [92]:
fib_gen = fibonnaci_generator()
for x in fib_gen:
    even_filter.send(x)
    if x > 200:
        break

1
1
2
2
3
5
8
8
13
21
34
34
55
89
144
144
233


A generator that can receive data like this is often referred to as a **coroutine**. Why coroutine? Because unlike regular subroutines (which run top-to-bottom and return once) they can pause, yield control, and be resumed later with external data, possibly by some external scheduler. That makes them perfect for **cooperative multitasking**, where functions can run concurrently by explicitly handing control back and forth, and sending values to each other, rather than stacking up on the call stack like traditional functions.

These generator-based coroutines were the first way to write asynchronous code in Python. Since version 3.5, Python introduced native coroutines using the `async def` and `await` keywords. Native coroutines are more ergonomic, but conceptually, they do the same thing: pause, wait for something, then resume.

In [93]:
from typing import Generator
import uuid


class Task:
    def __init__(self, coro: Generator):
        self.id = uuid.uuid4()
        self._coro = coro
        self.val = None

    def run(self):
        return self._coro.send(self.val)

# The val attribute represents the next value we want to send to the coroutine. We’ll use it to give
# back some result to the wrapped coroutine later on. After initialization, it’s None. Sending None to
# a coroutine is equivalent to calling next on it, and will advance to the next yield statement.

In [94]:
from collections import deque

class Scheduler:
    def __init__(self):
        self._queue = deque([])

    def create_task(self, coro: Generator):
        task = Task(coro)
        self._queue.append(task)

    def run_forever(self):
        while self._queue:
            task = self._queue.pop()
            try:
                task.run()
                self._queue.appendleft(task)
            except StopIteration as err:
                print(err.value)

About `run_forever()`:

This is as simple while loop that take the next task in the queue and run it once. Then either:
- We reschedule the task at the back of the queue
- StopIteration is raised, meaning the coroutine is over, and the task isn't rescheduled

💡 `StopIteration` is raised when a generator (or generator-style coroutine) finishes execution.
This happens either:
- When it reaches the end of the function, or
- When it hits a `return` statement (in which case the return value is stored in the exception’s `.value` attribute)

In [95]:
import time
def ping():
    count = 0
    while True:
        if count >= 5:
            return
        print("ping")
        time.sleep(.5)
        count += 1
        yield "ping done"


def pong():
    count = 0
    while True:
        if count >= 5:
            return
        print("pong")
        time.sleep(.5)
        count += 1
        yield "pong done"

scheduler = Scheduler()
scheduler.create_task(ping())
scheduler.create_task(pong())
scheduler.run_forever()

pong
ping
pong
ping
pong
ping
pong
ping
pong
ping
None
None


**Adding a Future**

A Future represents a value that’s not available yet, but will be eventually. That’s exactly what we need: we want our coroutine to yield a Future, and then have our scheduler “pause” it until that future is resolved.

In [96]:
from typing import Any, Callable
from abc import abstractmethod

class Future():
    def __init__(self):
        self._done_callbacks = []
        self.is_done = False
        self._result = None
        self._coro = self._run()

    def add_done_callback(self, callback: Callable[[Any], None]):
        self._done_callbacks.append(callback)

    def run(self):
        self._coro.send(None) # Equivalent to next(self._coro)

    def _run(self):
        while True:
            result = self._check_result()
            if result:
                self._done(result)
                return result
            yield

    def _done(self, result: Any):
        self.is_done = True
        self._result = result
        for callback in self._done_callbacks:
            callback(self._result)

    @abstractmethod
    def _check_result(self) -> Any:
        pass

# The Future has a run method, just like our Task. This is because the Future will
# replace the task in the queue, having a task-like interface allows us to treat it
# like a normal task.

In [97]:

from collections import deque
from typing import Generator

class Scheduler:
    def __init__(self):
        self._queue = deque([])

    def create_task(self, coro: Generator):
        task = Task(coro)
        self._schedule(task)

    def _schedule(self, task):
        self._queue.appendleft(task)

    def _continue(self, task):
        def callback(result):
            task.val = result
            self._schedule(task)

        return callback

    def run_forever(self):
        while self._queue:
            task = self._queue.pop()
            try:
                future = task.run()
                if isinstance(future, Future):
                    future.add_done_callback(self._continue(task))
                    self._schedule(future)
                else:
                    self._schedule(task)
            except StopIteration:
                pass

# Now when a task yields a Future, the scheduler sets a callback that:
#   1. Assigns the result back to the task (task.val)
#   2. Reschedules the task so it can resume execution

In [98]:
from datetime import datetime, timedelta, timezone

class Sleep(Future):
    def __init__(self, wait_for):
        super().__init__()
        self._wait_for = timedelta(seconds=wait_for)
        self._ready_at = datetime.now(tz=timezone.utc) + self._wait_for

    def _check_result(self):
        if datetime.now(tz=timezone.utc) >= self._ready_at:
            return True
        return None

In [ ]:
import time

def sleep(n: int):
    result = yield Sleep(n)
    return result

def ping():
    print("waiting for result before ping...")

    result = yield from sleep(5)
    # Without 'yield from', the equivalent code would be:
    # result = None
    # for s in sleep(5):
    #     result = yield s

    print(f"result: {result}")
    while True:
        print("ping")
        time.sleep(1)
        yield from sleep(0)

def pong():
    while True:
        print("pong")
        time.sleep(1)
        yield from sleep(0)

scheduler = Scheduler()
scheduler.create_task(ping())
scheduler.create_task(pong())
scheduler.run_forever()

waiting for result before ping...
ping waiting...
pong
pong
pong
pong
pong
result: True
ping
pong
ping
pong


KeyboardInterrupt: 

`ping()` pauses for 5 seconds, and during that time, pong can continue doing its thing. Just like with the real `asyncio.sleep()`.

The keywords `async` and `await` don't introduce new features to Python's execution model. They simply organize and improve what we've been doing with yield from and generators.

`await` works like `yield from`, and `async` alters our function type from Generator to Coroutine. This helps keep asynchronous generators/coroutines separate from regular generators.

To convert our generator-based coroutines into true coroutines, we just need to add `async` before them. Then we can use `await` with them (which is a drop-in replacement for `yield from`).

To integrate our custom Future with Python's await system, we need to make it awaitable. This means implementing the `__await__` method, which Python uses under the hood when you write await my_future.

In [112]:
from typing import Any, Callable, Generator
from abc import abstractmethod
from datetime import datetime, timedelta, timezone
from collections import deque

class Future_v2():
    def __init__(self):
        self._done_callbacks = []
        self.is_done = False
        self._result = None
        self._coro = self._run()
    
    def __await__(self):
        temp = yield self
        return temp

    def add_done_callback(self, callback: Callable[[Any], None]):
        self._done_callbacks.append(callback)

    def run(self):
        self._coro.send(None)

    def _run(self):
        while True:
            result = self._check_result()
            if result:
                self._done(result)
                return result
            yield

    def _done(self, result: Any):
        self.is_done = True
        self._result = result
        for callback in self._done_callbacks:
            callback(self._result)

    @abstractmethod
    def _check_result(self) -> Any:
        pass

class Scheduler_v2:
    def __init__(self):
        self._queue = deque([])

    def create_task(self, coro: Generator):
        task = Task(coro)
        self._schedule(task)

    def _schedule(self, task):
        self._queue.appendleft(task)

    def _continue(self, task):
        def callback(result):
            task.val = result
            self._schedule(task)

        return callback

    def run_forever(self):
        while self._queue:
            task = self._queue.pop()
            try:
                future = task.run()
                if isinstance(future, Future_v2):
                    future.add_done_callback(self._continue(task))
                    self._schedule(future)
                else:
                    self._schedule(task)
            except StopIteration:
                pass

class Sleep_v2(Future_v2):
    def __init__(self, wait_for):
        super().__init__()
        self._wait_for = timedelta(seconds=wait_for)
        self._ready_at = datetime.now(tz=timezone.utc) + self._wait_for

    def _check_result(self):
        if datetime.now(tz=timezone.utc) >= self._ready_at:
            return True
        return None

In [113]:
import time

async def sleep(n: int):
    result = await Sleep_v2(n)
    return result

async def ping():
    print("waiting for result before ping...")

    result = await sleep(5)

    print(f"result: {result}")
    while True:
        print("ping")
        time.sleep(1)
        await sleep(0)

async def pong():
    while True:
        print("pong")
        time.sleep(1)
        await sleep(0)

scheduler_v2 = Scheduler_v2()
scheduler_v2.create_task(ping())
scheduler_v2.create_task(pong())
scheduler_v2.run_forever()

waiting for result before ping...
pong
pong
pong
pong
pong
result: True
ping
pong
ping
pong
ping
pong


KeyboardInterrupt: 

/opt/homebrew/Cellar/python@3.13/3.13.7/Frameworks/Python.framework/Versions/3.13/lib/python3.13/re/__init__.py:310: RuntimeWarning: coroutine 'pong' was never awaited
  return pattern.translate(_special_chars_map)
